# Retrouver dans  Zenodo les jeux de données de son établissement 

In [1]:
import requests, json

In [2]:
# a gist example : https://gist.github.com/slint/d30fc0f415300876facbbeb8a0989ab2

l'API de Zenodo nécessite une clé. cf. https://developers.zenodo.org/#quickstart-upload

In [3]:
with open("../hide/personnal-keys.json") as f : 
    ACCESS_TOKEN = json.load(f)["ZENODO_KEY"]

## Défnir la recherche sur le champs affiliation

cf. https://help.zenodo.org/guides/search/

In [4]:
aff_query = "(univ* AND (diderot OR descartes))"

In [5]:
r = requests.get("https://zenodo.org/api/records",
                 params = {"q" : "creators.affiliation:" +  aff_query,
                           "type" : "dataset",
                           "size" : 300,
                           "sort" : "mostrecent",
                           "all_version" : False,
                           "access_tpoken" : ACCESS_TOKEN
                          })
res = r.json()

Se familiariser avec la structure de la réponse

In [6]:
# export les résultats en JSON dans un fichier local "zenodo_raw.json"
# ouvrir ce fichier en local et comprends sa structure
with open("zenodo_raw.json", 'w') as f : 
    json.dump(res, f)

In [9]:
# imprimer le premier résultat de la requête
print(json.dumps(res["hits"]["hits"][1],  indent = 1))

{
 "conceptdoi": "10.5281/zenodo.6808400",
 "conceptrecid": "6808400",
 "created": "2022-08-15T11:32:14.492184+00:00",
 "doi": "10.5281/zenodo.6808401",
 "files": [
  {
   "bucket": "f603f45b-2b30-4f8a-8dc1-0e1c4afc8b96",
   "checksum": "md5:775688570203529fd335d7899fbf7390",
   "key": "Micropollutants data and code for urban wet-weather flows.zip",
   "links": {
    "self": "https://zenodo.org/api/files/f603f45b-2b30-4f8a-8dc1-0e1c4afc8b96/Micropollutants%20data%20and%20code%20for%20urban%20wet-weather%20flows.zip"
   },
   "size": 3434073,
   "type": "zip"
  }
 ],
 "id": 6808401,
 "links": {
  "badge": "https://zenodo.org/badge/doi/10.5281/zenodo.6808401.svg",
  "bucket": "https://zenodo.org/api/files/f603f45b-2b30-4f8a-8dc1-0e1c4afc8b96",
  "conceptbadge": "https://zenodo.org/badge/doi/10.5281/zenodo.6808400.svg",
  "conceptdoi": "https://doi.org/10.5281/zenodo.6808400",
  "doi": "https://doi.org/10.5281/zenodo.6808401",
  "html": "https://zenodo.org/record/6808401",
  "latest": "ht

In [10]:
# nb de jeux de données trouvés
print(res["hits"]["total"])

72


In [ ]:
# imprimer le 10e résultat de la requête
print(json.dumps(res["hits"]["hits"][10], indent = 2))

In [11]:
# imprimer les clés d'un hits
first_hit = res["hits"]["hits"][0]
first_hit.keys()

dict_keys(['conceptdoi', 'conceptrecid', 'created', 'doi', 'files', 'id', 'links', 'metadata', 'owners', 'revision', 'stats', 'updated'])

## Déduire les informations à récupérer
``` 
id, l'identifiant zenodo, pour déduire le lien https://zenodo.org/record/5005868
metadata/publication_date : date du dépot
metadata/titre : titre du jeu de de donnée
metadata/license : la licence apposée
metadata/keywords
metadata/doi
```

## Récupérer les données

In [12]:
# un dictionnaire pour récupérer les données
data = {}

# itérer sur les résultats et placer les données souhaitées dans le dictionnaire data
for idx, item in enumerate( res["hits"]["hits"] ) : 
    
    data[idx] = {}
    data[idx]["id"] = item["id"]
    
    data[idx]["title"] = item["metadata"].get("title")
    data[idx]["publication_date"] = item["metadata"].get("publication_date")
    data[idx]["doi"] = item["metadata"].get("doi")
    
    #licence
    if item["metadata"].get("license") : 
        data[idx]["license"] = item["metadata"]["license"]["id"]   
    
    # mots-clés
    if item["metadata"].get("keywords") : 
        data[idx]["keywords"] = ",".join(item["metadata"]["keywords"])  

In [13]:
## imprimer les données récupérer d'un jeu de donnée
print( json.dumps(data[3], indent = 1))

{
 "id": 5005868,
 "title": "A global inventory of solar photovoltaic generating units - dataset",
 "publication_date": "2021-10-27",
 "doi": "10.5281/zenodo.5005868",
 "license": "CC-BY-4.0",
 "keywords": "photovoltaic solar,remote sensing,geospatial data,computer vision"
}


## Placer les données dans un dataframe

In [14]:
import pandas as pd

In [27]:
df = pd.DataFrame.from_dict(data, orient='index')

In [29]:
df.sample(2)

,id,title,publication_date,doi,license,keywords
13,5003345,Data from: State transitions: a major mortalit...,2018-05-23,10.5061/dryad.km2s0,CC0-1.0,"primate,Microcebus murinus,Biological rhythm,P..."
16,4999955,Data from: Molecular determinants of agonist s...,2015-08-26,10.5061/dryad.k47p7,CC0-1.0,NaN


In [30]:
# trier la table par date de dépôt
df = df.sort_values( by = "publication_date", ascending = False)

In [31]:
df[:4]

,id,title,publication_date,doi,license,keywords
0,7104630,"Experimental data of ""Mesoscopic Klein-Schwing...",2022-09-22,10.5281/zenodo.7104630,CC-BY-4.0,NaN
1,6808401,"Data and code for ""A decade of monitoring micr...",2022-08-09,10.5281/zenodo.6808401,GPL-3.0,"micropollutants,wet-weather flows,stormwater,m..."
2,6760262,Molecular and electrophysiological features of...,2022-06-27,10.5061/dryad.37pvmcvp1,CC0-1.0,"classification,hippocampus,Interneurons,Dentat..."
3,5005868,A global inventory of solar photovoltaic gener...,2021-10-27,10.5281/zenodo.5005868,CC-BY-4.0,"photovoltaic solar,remote sensing,geospatial d..."


In [ ]:
## exporter la table en CSV
df.to_csv("data_from_zenodo.csv", index = False)

----
## Vigilance

* la recherche dans l'affiliation se fait pour tous les auteurs (comme le Wos) donc du bruit peut être présent en sortie

* l'API donne les résultats avec un systeme de pagination. Vérifier à la fin de la réponse si `links` ne contient pas une url `next`